# Problema 4 - Geração de Texto com RNN, LSTM e GRU

Este notebook tem como objetivo comparar o desempenho de três arquiteturas de redes neurais recorrentes (`SimpleRNN`, `LSTM` e `GRU`) em uma tarefa de **geração de texto caractere por caractere**.

## Objetivo
- Treinar os três modelos para aprender a estrutura da linguagem a partir dos livros da série Harry Potter.
- Gerar texto com cada um dos modelos para comparar visualmente a sua capacidade de criar palavras e frases coerentes.
- Discutir as diferenças de performance e eficiência entre `SimpleRNN`, `LSTM` e `GRU`.

## Dataset
Utilizaremos os três primeiros livros da série Harry Potter em português. Os modelos aprenderão a prever o próximo caractere de uma sequência, e usaremos esse poder preditivo para gerar novos textos.

## 1. Importação das Bibliotecas

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, GRU, Embedding
from tensorflow.keras.utils import to_categorical
import os
import time

# Configurar seed para reprodutibilidade
np.random.seed(42)
tf.random.set_seed(42)

import tensorflow as tf
from tensorflow.keras import mixed_precision

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.list_physical_devices('CPU')))
print("TensorFlow version:", tf.__version__)

mixed_precision.set_global_policy('mixed_float16')

## 2. Carregamento e Preparação dos Dados

Vamos carregar os textos dos 3 livros, juntá-los em um único corpus e criar o nosso vocabulário de caracteres.

In [ ]:
# Carregar e concatenar os textos
text = ""
for i in range(1, 4):
    filepath = os.path.join('dataset', f'harry_potter_{i}.txt')
    with open(filepath, 'r', encoding='utf-8') as f:
        text += f.read()

# Converter para minúsculas para reduzir o vocabulário
text = text.lower()

# Criar o vocabulário de caracteres
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"Tamanho do corpus: {len(text)} caracteres")
print(f"Tamanho do vocabulário: {vocab_size} caracteres")
print(f"Vocabulário: {''.join(chars)}")

# Criar dicionários de mapeamento
char_to_int = {c: i for i, c in enumerate(chars)}
int_to_char = {i: c for i, c in enumerate(chars)}

## 3. Criação das Sequências de Treino

Agora, vamos transformar nosso longo texto em pares de `(entrada, saída)` para treinar os modelos. Usaremos uma janela deslizante para criar as sequências.

- **Tamanho da Sequência (`seq_length`):** 100 caracteres.
- **Entrada (`X`):** Uma sequência de 100 caracteres.
- **Saída (`y`):** O 101º caractere.

In [ ]:
seq_length = 100
X_data = []
y_data = []

for i in range(0, len(text) - seq_length, 1):
    # Sequência de entrada
    in_seq = text[i:i + seq_length]
    # Caractere de saída
    out_char = text[i + seq_length]
    # Armazenar como inteiros
    X_data.append([char_to_int[char] for char in in_seq])
    y_data.append(char_to_int[out_char])

n_patterns = len(X_data)
print(f"Total de sequências de treino: {n_patterns}")

# Preparar os dados para a rede neural
X = np.reshape(X_data, (n_patterns, seq_length, 1))
# Normalizar
X = X / float(vocab_size)
# One-hot encode da variável de saída
y = to_categorical(y_data)

## 4. Construção e Treinamento dos Modelos

Vamos criar uma função para construir os modelos, pois a arquitetura será muito semelhante, mudando apenas a camada recorrente. Em seguida, treinaremos cada um deles.

**Atenção:** O treinamento pode ser demorado. Para uma demonstração, poucas épocas já são suficientes para ver a diferença de performance.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import gc

def create_model(recurrent_layer, vocab_size, seq_length):
    model = Sequential([
        recurrent_layer(128, input_shape=(seq_length, 1), return_sequences=True),
        recurrent_layer(128),
        Dense(vocab_size, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def generate_text(model, seed_text, num_chars_to_gen=200):
    # Converter a semente para minúsculas
    full_generated_text = seed_text.lower()
    
    # Preparar a semente inicial para ter o tamanho correto (seq_length)
    # Se a semente for maior, pegamos os últimos 'seq_length' caracteres.
    # Se for menor, preenchemos com espaços à esquerda.
    current_pattern_text = full_generated_text.rjust(seq_length)
    
    # Converter o padrão inicial para inteiros
    pattern = [char_to_int[char] for char in current_pattern_text[-seq_length:]]
    
    print(f"Semente: \"{seed_text}\"")
    print("Texto gerado:")
    print("------------------")
    print(seed_text, end='') # Imprime a semente original
    
    for i in range(num_chars_to_gen):
        # Preparar entrada para o modelo (sempre com seq_length)
        x = np.reshape(pattern, (1, seq_length, 1))
        x = x / float(vocab_size)
        
        # Fazer a previsão
        prediction = model.predict(x, verbose=0)
        
        # Pegar o caractere com a maior probabilidade
        index = np.argmax(prediction)
        result = int_to_char[index]
        
        # Adicionar ao texto gerado e atualizar o padrão
        full_generated_text += result
        pattern.append(index)
        pattern = pattern[1:len(pattern)] # Mantém o padrão com o tamanho de seq_length
        
        # Imprimir o caractere gerado
        print(result, end='')
        
    print("\n------------------\n")

# Configurar callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',          # Métrica a monitorar
    patience=2,                  # Parar após 2 épocas sem melhoria (era 3)
    min_delta=0.001,             # Considerar melhora apenas se for > 0.001
    restore_best_weights=True,   # Restaurar os melhores pesos
    verbose=1                    # Mostrar quando parar
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',          # Métrica a monitorar
    factor=0.5,                  # Reduzir LR pela metade
    patience=1,                  # Reduzir LR após 1 época sem melhora (era 2)
    min_lr=1e-6,                 # LR mínimo
    verbose=1
)

callbacks = [early_stopping, reduce_lr]
epochs = 20
seed = "harry potter olhou para o castelo e"

In [ ]:
# --- Modelo 1: SimpleRNN ---
print("### Treinando o Modelo SimpleRNN ###")
rnn_model = create_model(SimpleRNN, vocab_size, seq_length)
rnn_model.fit(X, y, epochs=epochs, batch_size=128, verbose=1, validation_split=0.1, callbacks=callbacks)

print("\n--- Geração com SimpleRNN ---")
generate_text(rnn_model, seed)

# Limpar memória
del rnn_model
tf.keras.backend.clear_session()
gc.collect()

In [ ]:
# --- Modelo 2: LSTM ---
print("\n### Treinando o Modelo LSTM ###")
lstm_model = create_model(LSTM, vocab_size, seq_length)
lstm_model.fit(X, y, epochs=epochs, batch_size=128, verbose=1, validation_split=0.1, callbacks=callbacks)

print("\n--- Geração com LSTM ---")
generate_text(lstm_model, seed)

# Limpar memória
del lstm_model
tf.keras.backend.clear_session()
gc.collect()

In [ ]:
# --- Modelo 3: GRU ---
print("\n### Treinando o Modelo GRU ###")
gru_model = create_model(GRU, vocab_size, seq_length)
gru_model.fit(X, y, epochs=epochs, batch_size=128, verbose=1, validation_split=0.1, callbacks=callbacks)

print("\n--- Geração com GRU ---")
generate_text(gru_model, seed)

# Limpar memória
del gru_model
tf.keras.backend.clear_session()
gc.collect()